In [1]:
import os

In [2]:
os.getenv("PATH")

'/cluster/project/treutlein/jjans/software/miniforge3/envs/chrombpnet2/bin:/cluster/apps/lsf/10.1/linux2.6-glibc2.3-x86_64/bin:/cluster/apps/lsf/10.1/linux2.6-glibc2.3-x86_64/bin:/cluster/apps/gcc-8.2.0/cuda-12.1.1-mpwcqkwqghc7y2at5a6wuuhbgmm6efux/bin:/cluster/apps/gcc-8.2.0/openblas-0.3.15-huwxbhezdzoo74awrgoz6sd2qndpmdva/bin:/cluster/apps/nss/gcc-8.2.0/python/3.10.4/x86_64/bin:/cluster/apps/gcc-8.2.0/eccodes-2.21.0-o4xitaateyj4fuopb6chuxme7d5bp4zp/bin:/cluster/apps/gcc-8.2.0/hdf5-1.10.1-qj3ju3qfhvucsk5eevrtb2lehbux5nmv/bin:/cluster/apps/nss/jupyterhub/3.5.1/bin:/cluster/apps/gcc-8.2.0/git-2.31.1-q45wg6avfyvko4weuhmnpghaag45ynoo/bin:/cluster/apps/gcc-8.2.0/npm-6.14.9-774crfohwvu6a33ijcow7x5cvonu44oi/bin:/cluster/apps/gcc-8.2.0/r-4.2.2-ydfaklhfrhw5dy6qcfzxlxfviwovcord/bin:/cluster/spack/apps/linux-centos7-x86_64/gcc-4.8.5/gcc-8.2.0-6xqov2fhvbmehix42slain67vprec3fs/bin:/cluster/apps/local:/cluster/apps/sfos/bin:/usr/local/bin:/usr/local/sbin:/usr/sbin:/usr/bin:/sbin:/bin:/cluster/slurm/

In [3]:
os.getenv("LD_LIBRARY_PATH")

'/cluster/project/treutlein/jjans/software/miniforge3/envs/cuda11_env/lib:/cluster/project/treutlein/jjans/software/miniforge3/envs/chrompbnet2/lib:/cluster/project/treutlein/jjans/software/miniforge3/envs/chrompbnet/lib:/cluster/apps/lsf/10.1/linux2.6-glibc2.3-x86_64/lib:/cluster/apps/lsf/10.1/linux2.6-glibc2.3-x86_64/lib:/cluster/apps/gcc-8.2.0/cudnn-8.9.2.26-ogi7ed2h6ejs7vumekv46idqqas4axgq/lib:/cluster/apps/gcc-8.2.0/cuda-12.1.1-mpwcqkwqghc7y2at5a6wuuhbgmm6efux/lib64:/cluster/apps/gcc-8.2.0/nccl-2.11.4-1-pwkiz23vbeac3vt5ykybdwzaykprizb2/lib:/cluster/apps/gcc-8.2.0/openblas-0.3.15-huwxbhezdzoo74awrgoz6sd2qndpmdva/lib:/cluster/apps/nss/gcc-8.2.0/python/3.10.4/x86_64/lib64:/cluster/apps/gcc-8.2.0/zlib-1.2.9-roj3c3p7lbd2kn3gstlt4rxdcgvb3csi/lib:/cluster/apps/gcc-8.2.0/eccodes-2.21.0-o4xitaateyj4fuopb6chuxme7d5bp4zp/lib64:/cluster/apps/gcc-8.2.0/hdf5-1.10.1-qj3ju3qfhvucsk5eevrtb2lehbux5nmv/lib:/cluster/apps/gcc-8.2.0/npm-6.14.9-774crfohwvu6a33ijcow7x5cvonu44oi/lib:/cluster/apps/gcc-8.2.

In [4]:
# Adapted from chrombpnet-lite

import deepdish as dd
import json
import numpy as np
import tensorflow as tf
import pandas as pd
import shap
import pyfaidx
import shutil
import errno
import os


In [5]:
import argparse
import chrombpnet.evaluation.interpret.shap_utils as shap_utils
import chrombpnet.evaluation.interpret.input_utils as input_utils

NARROWPEAK_SCHEMA = ["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"]


In [6]:
# disable eager execution so shap deep explainer wont break
tf.compat.v1.disable_eager_execution()


In [7]:
def fetch_interpret_args():
    parser = argparse.ArgumentParser(description="get sequence contribution scores for the model")
    parser.add_argument("-g", "--genome", type=str, required=True, help="Genome fasta")
    parser.add_argument("-r", "--regions", type=str, required=True, help="10 column bed file of peaks. Sequences and labels will be extracted centered at start (2nd col) + summit (10th col).")
    parser.add_argument("-m", "--model_h5", type=str, required=True, help="Path to trained model, can be both bias or chrombpnet model")
    parser.add_argument("-o", "--output-prefix", type=str, required=True, help="Output prefix")
    parser.add_argument("-d", "--debug_chr", nargs="+", type=str, default=None, help="Run for specific chromosomes only (e.g. chr1 chr2) for debugging")
    parser.add_argument("-p", "--profile_or_counts", nargs="+", type=str, default=["counts", "profile"], choices=["counts", "profile"],
                        help="use either counts or profile or both for running shap")

    args = parser.parse_args()
    return args


In [8]:
def generate_shap_dict(seqs, scores):
    assert(seqs.shape==scores.shape)
    assert(seqs.shape[2]==4)

    # construct a dictionary for the raw shap scores and the
    # the projected shap scores
    # MODISCO workflow expects one hot sequences with shape (None,4,inputlen)
    d = {
            'raw': {'seq': np.transpose(seqs, (0, 2, 1)).astype(np.int8)},
            'shap': {'seq': np.transpose(scores, (0, 2, 1)).astype(np.float16)},
            'projected_shap': {'seq': np.transpose(seqs*scores, (0, 2, 1)).astype(np.float16)}
        }

    return d


In [9]:
def interpret(model, seqs, output_prefix, profile_or_counts,save_files=True):
    print("Seqs dimension : {}".format(seqs.shape))

    outlen = model.output_shape[0][1]

    profile_model_input = model.input
    profile_input = seqs
    counts_model_input = model.input
    counts_input = seqs

    if "counts" in profile_or_counts:
        profile_model_counts_explainer = shap.explainers.deep.TFDeepExplainer(
            (counts_model_input, tf.reduce_sum(model.outputs[1], axis=-1)),
            shap_utils.shuffle_several_times,
            combine_mult_and_diffref=shap_utils.combine_mult_and_diffref)

        print("Generating 'counts' shap scores")
        counts_shap_scores = profile_model_counts_explainer.shap_values(
            counts_input, progress_message=100)

        counts_scores_dict = generate_shap_dict(seqs, counts_shap_scores)

        if save_files:
            # save the dictionary in HDF5 formnat
            print("Saving 'counts' scores")

            dd.io.save("{}.counts_scores.h5".format(output_prefix),
                        counts_scores_dict,
                        compression='blosc')

            del counts_shap_scores, counts_scores_dict

    if "profile" in profile_or_counts:
        weightedsum_meannormed_logits = shap_utils.get_weightedsum_meannormed_logits(model)
        profile_model_profile_explainer = shap.explainers.deep.TFDeepExplainer(
            (profile_model_input, weightedsum_meannormed_logits),
            shap_utils.shuffle_several_times,
            combine_mult_and_diffref=shap_utils.combine_mult_and_diffref)

        print("Generating 'profile' shap scores")
        profile_shap_scores = profile_model_profile_explainer.shap_values(
            profile_input, progress_message=100)

        profile_scores_dict = generate_shap_dict(seqs, profile_shap_scores)

        if save_files:
            # save the dictionary in HDF5 formnat
            print("Saving 'profile' scores")
            dd.io.save("{}.profile_scores.h5".format(output_prefix),
                        profile_scores_dict,
                        compression='blosc')
    
    results = {}
    results['profile'] = ''
    results['counts_scores'] = ''
    results['counts_shap'] = ''
    if "profile" in profile_or_counts:
        results['profile'] = profile_scores_dict
    if "counts" in profile_or_counts:
        results['counts_scores'] = counts_scores_dict
        results['counts_shap'] = counts_shap_scores

    return(results)


In [10]:
NARROWPEAK_SCHEMA = ["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"]

regions_df = pd.read_csv("regions_of_interest/IGFBP2_regions_peaks_human.bed", sep='\t', names=NARROWPEAK_SCHEMA)


In [11]:
regions_df

,chr,start,end,1,2,3,4,5,6,summit
0,chr2,216636508,216637488,human_peaks_peak_48293,380,.,5.29416,40.7695,38.0041,799
1,chr2,216863820,216864761,human_peaks_peak_48308,340,.,7.94451,36.7715,34.0687,362
2,chr2,216863820,216864761,human_peaks_peak_48308,340,.,7.94451,36.7715,34.0687,362


In [12]:
import os
import sys
import os
import sys
import tensorflow as tf
import numpy as np
import chrombpnet.training.utils.losses as losses
from chrombpnet.training.utils.data_utils import one_hot
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.models import load_model


def get_seq(peaks_df, genome, width):
    """
    Same as get_cts, but fetches sequence from a given genome.
    """
    vals = []
    peaks_used = []
    for i, r in peaks_df.iterrows():
        sequence = str(genome[r['chr']][(r['start']+r['summit'] - width//2):(r['start'] + r['summit'] + width//2)])
        if len(sequence) == width:
            vals.append(sequence)
            peaks_used.append(True)
        else:
            peaks_used.append(False)

    return one_hot.dna_to_one_hot(vals), np.array(peaks_used)



In [13]:
for fold in ['fold_0','fold_1','fold_2','fold_3','fold_4']:
    
    model_path = 'celltype_models_human/modelv1_enterocytes/'+fold+'/output/models/chrombpnet_nobias.h5'
    
    custom_objects={"multinomial_nll": losses.multinomial_nll, "tf": tf}    
    get_custom_objects().update(custom_objects)    
    model=load_model(model_path,compile=False)
    print("got the model")

got the model
got the model
got the model
got the model
got the model


In [14]:
# infer input length
inputlen = model.input_shape[1] # if bias model (1 input only)
print("inferred model inputlen: ", inputlen)

inferred model inputlen:  2114


In [15]:
genome_path = 'encode_data/hg38.fa'
region_index = 1
#FILL IN DETAILS FROM REGION_DF 

import pyfaidx
genome_path = 'encode_data/hg38.fa'
genome = pyfaidx.Fasta(genome_path)
width = input_len = inputlen
chrom=regions_df.loc[region_index,'chr']
start,end = regions_df.loc[region_index,'start'],regions_df.loc[region_index,'end']
summit = regions_df.loc[region_index,'summit']

start_use = int(start+summit - input_len/2)+1
end_use = int(start + summit + input_len/2)

middle = int((start+end)/2)
nstart = middle-int(input_len/2)
nend = nstart + input_len

#fasta format
seq_man = genome.get_seq(chrom,start_use,end_use)

#string
seq_man2 = str(genome[chrom][(start_use-1):(end_use)])


sequence = seq_man2
vals = []
if len(sequence) == width:
    vals.append(sequence)


In [16]:
def evo_seq(sequence,position,ref,alt):
    if sequence[position]==ref:
        mod_seq = sequence[:position] + alt + sequence[position + 1:]
    else:
        print("reference does not occur at position, double check position")
        mod_seq = None
    return(mod_seq)

def softmax(x, temp=1):
    norm_x = x - np.mean(x,axis=1, keepdims=True)
    return np.exp(temp*norm_x)/np.sum(np.exp(temp*norm_x), axis=1, keepdims=True)



In [17]:
# Function to generate SNPs

def generate_snps(sequence):
    import pandas as pd
    snp_list_df = []
    snp_list = []
    ref_list = []
    # List of possible bases
    bases = ['A', 'C', 'G', 'T']
    for i in range(len(sequence)):
        for base in bases:
            if sequence[i] != base:
                mutated_sequence = sequence[:i] + base + sequence[i+1:]
                snp_list.append(mutated_sequence)
                ref_list.append(sequence)
                snp_list_df.append(dict(pos=i,ref=sequence[i],alt=base))
    snp_list_df = pd.DataFrame(snp_list_df)
    return snp_list,snp_list_df,ref_list


In [18]:
sequence_ISM_list,sequence_ISM_df,ref_sequences = generate_snps(sequence)

In [19]:
sequence_ISM_df

,pos,ref,alt
0,0,A,C
1,0,A,G
2,0,A,T
3,1,C,A
4,1,C,G
...,...,...,...
6337,2112,G,C
6338,2112,G,T
6339,2113,T,A
6340,2113,T,C


In [20]:
all_mutated_sequences_ohe = one_hot.dna_to_one_hot(sequence_ISM_list)
ref_sequences_ohe = one_hot.dna_to_one_hot(ref_sequences)

In [21]:
import time

In [22]:
import random
random.seed(42)

In [23]:
all_mutated_sequences_df = sequence_ISM_df.copy()

In [24]:
for fold in ['fold_0','fold_1','fold_2','fold_3','fold_4']:
    print(fold)
    model_path = 'celltype_models_human/modelv1_enterocytes/'+fold+'/output/models/chrombpnet_nobias.h5'
    
    custom_objects={"multinomial_nll": losses.multinomial_nll, "tf": tf}    
    get_custom_objects().update(custom_objects)    
    model=load_model(model_path,compile=False)
    print("got the model")

    ref_logcount_preds=[]
    alt_logcount_preds=[]
    ref_prob_preds=[]
    alt_prob_preds=[]

    print("predict reference sequence")
    ref_batch_preds = model.predict(ref_sequences_ohe)
    time.sleep(5)

    print("predict mutations sequence")
    alt_batch_preds = model.predict(all_mutated_sequences_ohe)
    time.sleep(5)

    ref_logcount_preds.extend(np.squeeze(ref_batch_preds[1]))
    alt_logcount_preds.extend(np.squeeze(alt_batch_preds[1]))
    
    ref_prob_preds.extend(np.squeeze(softmax(ref_batch_preds[0])))
    alt_prob_preds.extend(np.squeeze(softmax(alt_batch_preds[0])))

    ref_logcount_preds = np.array(ref_logcount_preds)
    alt_logcount_preds = np.array(alt_logcount_preds)
    ref_prob_preds = np.array(ref_prob_preds)
    alt_prob_preds = np.array(alt_prob_preds)

    from scipy.spatial.distance import jensenshannon
    
    log_counts_diff = alt_logcount_preds - ref_logcount_preds
    log_probs_diff_abs_sum =  np.sum(np.abs(np.log(alt_prob_preds) -  np.log(ref_prob_preds)),axis=1)*np.sign(log_counts_diff)
    probs_jsd_diff = np.array([jensenshannon(x,y) for x,y in zip(alt_prob_preds, ref_prob_preds)])*np.sign(log_counts_diff)

    all_mutated_sequences_df[fold+'--logcount_preds'] = alt_logcount_preds
    all_mutated_sequences_df[fold+'--log_counts_diff'] = log_counts_diff
    all_mutated_sequences_df[fold+'--log_probs_diff_abs_sum'] = log_probs_diff_abs_sum
    all_mutated_sequences_df[fold+'--probs_jsd_diff'] = probs_jsd_diff

fold_0
got the model
predict reference sequence


/cluster/home/jjanssens/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


predict mutations sequence


/cluster/home/jjanssens/.local/lib/python3.10/site-packages/scipy/spatial/distance.py:1272: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)


fold_1
got the model
predict reference sequence


...

/cluster/home/jjanssens/.local/lib/python3.10/site-packages/scipy/spatial/distance.py:1272: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)


fold_4
got the model
predict reference sequence


/cluster/home/jjanssens/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


predict mutations sequence


/cluster/home/jjanssens/.local/lib/python3.10/site-packages/scipy/spatial/distance.py:1272: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)


In [25]:
log_counts_diff_columns = [x for x in all_mutated_sequences_df.columns if 'log_counts_diff' in x]
log_probs_diff_abs_sum_columns = [x for x in all_mutated_sequences_df.columns if 'log_probs_diff_abs_sum' in x]
probs_jsd_diff_columns = [x for x in all_mutated_sequences_df.columns if 'probs_jsd_diff' in x]
logcount_preds_columns = [x for x in all_mutated_sequences_df.columns if 'logcount_preds' in x]

In [26]:
all_mutated_sequences_df['log_counts_diff_avg'] = all_mutated_sequences_df[log_counts_diff_columns].T.mean()
all_mutated_sequences_df['logcount_preds_avg'] = all_mutated_sequences_df[logcount_preds_columns].T.mean()

all_mutated_sequences_df['log_probs_diff_abs_sum_avg'] = all_mutated_sequences_df[log_probs_diff_abs_sum_columns].T.mean()
all_mutated_sequences_df['probs_jsd_diff_avg'] = all_mutated_sequences_df[probs_jsd_diff_columns].T.mean()

In [27]:
all_mutated_sequences_df.to_csv("IGFBP2_distal_ISM_study_effects_wcounts.tsv",sep="\t")